In [120]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util import Retry
from selenium import webdriver
from selenium.webdriver.support.select import Select
import csv
import requests
import random
import time
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [44]:
longDelay_choices = [10, 7, 21, 16, 13, 22, 19, 24, 12, 11]
longDelay = random.choice(longDelay_choices)
shortDelay_choices = [8, 5, 3, 7]
shortDelay = random.choice(shortDelay_choices)

# HTML parsing function

In [45]:
def get_data(soup):
    d = dict()
    for i in soup.find("table", {"id": "format0_disparea"}).findAll("tr"):
        try:
            key = i.find("th", {"class": "std1"}).text
            try:
                link = i.find("input", {"id": "fe_text1"})["value"]
                d[key] = link
            except:
                value = i.find("td", {"class": "std2"}).text
                d[key] = value
        except:
            pass
        
    return d

# CSV export function

In [80]:
def export_csv(professor, r1, dic):
    filename = professor+str(r1)
    filepath = "/Users/chenxi/Documents/SideProjects/These_scraper/export/{}.csv".format(filename)
    with open(filepath, "w") as file:
        fieldnames = list(key for key in dic.keys())
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(dic)
    file.close()

# Selenium function

In [84]:
def search_prof(chrome, professor):
    selectProfessor = chrome.find_element_by_id("ad_指導教授")
    selectProfessor.click()
    #time.sleep(shortDelay)
    inputBox = chrome.find_element_by_id("ysearchinput0")
    inputBox.send_keys(professor)
    #time.sleep(shortDelay)
    search = chrome.find_element_by_id("gs32search")
    search.click()
    time.sleep(shortDelay)

In [83]:
def get_Urls_by_pages(chrome, hrefList, page):
    print("Jump to page {}".format(page))
    chrome.find_element_by_id("jmpage").clear()
    chrome.find_element_by_id("jmpage").send_keys(page)
    time.sleep(shortDelay)
    goToPage = chrome.find_element_by_name("jumpfmt1page")
    goToPage.click()
    print("Find all links")
    time.sleep(shortDelay)
    thesisLinks = chrome.find_elements_by_class_name("slink")
    
    for link in thesisLinks:
        print("Get link {} into href".format(link))
        hrefList.append(link.get_attribute("href"))

In [87]:
def crawl_prof_thesis_by_pages(professor, pages):
    page = 1
    index = 1
    while page <= pages:
        print("Crawl page {}".format(page))
        hrefList = []
        chrome = webdriver.Chrome("./chromedriver")
        chrome.get("https://ndltd.ncl.edu.tw/")
        #time.sleep(longDelay)
        search_prof(chrome, professor)
        get_Urls_by_pages(chrome, hrefList, page)
        
        print("Export href into csv")
        for href in hrefList:
            chrome.get(href)
            soup = BeautifulSoup(chrome.page_source, 'html.parser')
            thesisDict = dict()
            thesisDict = get_data(soup)
            export_csv("王惠嘉", index, thesisDict)
            index += 1
            
        chrome.close()
        page += 1

# Crawling data

In [88]:
crawl_prof_thesis_by_pages("王惠嘉", 3)

Crawl page 1
Jump to page 1
Find all links
Get link <selenium.webdriver.remote.webelement.WebElement (session="97ccdab7bf9d74622e43661ace71edff", element="1e81f6c8-423a-4261-b233-1630a9046ca6")> into href
Get link <selenium.webdriver.remote.webelement.WebElement (session="97ccdab7bf9d74622e43661ace71edff", element="a781afe5-8db1-409d-bf71-d5b841b692f9")> into href
Get link <selenium.webdriver.remote.webelement.WebElement (session="97ccdab7bf9d74622e43661ace71edff", element="f1efb35b-4aff-48f7-80b7-35a63b2a3371")> into href
Get link <selenium.webdriver.remote.webelement.WebElement (session="97ccdab7bf9d74622e43661ace71edff", element="4dd9b3f3-99b3-480f-8ef5-4ebe107837f2")> into href
Get link <selenium.webdriver.remote.webelement.WebElement (session="97ccdab7bf9d74622e43661ace71edff", element="38a8519d-0905-4354-a3b6-4c56c25a347a")> into href
Get link <selenium.webdriver.remote.webelement.WebElement (session="97ccdab7bf9d74622e43661ace71edff", element="60a92ecd-fa57-4354-8764-f1a0857b33e

# Merging CSV files

In [115]:
def merge_csv_by_prof(professor):
    filepath = "/Users/chenxi/Documents/SideProjects/These_scraper/export/{}*.csv".format(professor)
    files = glob(filepath)
    df = pd.concat((pd.read_csv(file) for file in files), join='outer')
    df.index = np.arange(1, len(df)+1)
    print(df)
    
    filename = "/Users/chenxi/Documents/SideProjects/These_scraper/export/{}.csv".format(professor)
    df.to_csv(filename)

In [116]:
merge_csv_by_prof("王惠嘉")

                               本論文永久網址: 研究生:         研究生(外文):  \
1   https://hdl.handle.net/11296/ycdyf9  鄭人瑋     Jen-WeiCheng   
2   https://hdl.handle.net/11296/f32b28  徐尚煒     Hsang-WeiHsu   
3   https://hdl.handle.net/11296/w52u2k  劉如祥     Ru-SiangLiou   
4   https://hdl.handle.net/11296/5s69fw  李亭葦       Ting-WeiLi   
5   https://hdl.handle.net/11296/9e8723  徐陞瑋     Sheng-WeiSyu   
6   https://hdl.handle.net/11296/9xqp7e  林思婷       Si-TingLin   
7   https://hdl.handle.net/11296/p6ch5t  楊穗結     Suei-JieYang   
8   https://hdl.handle.net/11296/g6m8wb  卿雯鈞    Wen-JiunChing   
9   https://hdl.handle.net/11296/2yy32c  趙雅蘭       Ya-LanChao   
10  https://hdl.handle.net/11296/y95yu2  王丞宇     Cheng-YuWang   
11  https://hdl.handle.net/11296/r8c7xc  陳怡帆        I-FanChen   
12  https://hdl.handle.net/11296/y932v9  林彥廷      Yen-TingLin   
13  https://hdl.handle.net/11296/ej57nz  程家浩   Cheng, Jia-Hao   
14  https://hdl.handle.net/11296/56gvp7  吳佳穎    Wu, Chia-Ying   
15  https://hdl.handle.ne

# Analysis

### 論文關鍵字次數


In [ ]:
def split_value(df, column):
    for row in df[column]:
        

In [150]:
filepath = "/Users/chenxi/Documents/SideProjects/These_scraper/export/{}.csv".format("王惠嘉")
df = pd.read_csv(filepath)
keywords = df["中文關鍵詞:"].str.split('、', expand=True)
keywords = keywords.stack()
print(keywords.value_counts())

文字探勘                   4
主題模型                   3
資料探勘                   2
深度學習                   2
問答系統                   2
自動擷取摘要                 1
移動行為                   1
旅遊推薦                   1
筆記摘要                   1
主題識別                   1
SWOT                   1
服務組合                   1
QoS                    1
執法與服務品質管理              1
議題追蹤系統                 1
議題分類                   1
引用推薦                   1
語意分析                   1
文件分群                   1
文字摘要                   1
In-flipped Learning    1
混合式學習                  1
大學教室環境量表               1
主題地圖                   1
社群資訊                   1
滑動視窗                   1
積淹水                    1
意見領袖                   1
情感分析                   1
趨勢分析                   1
電子票證                   1
特徵選取                   1
資訊視覺化                  1
引用句偵測                  1
空間向量模型                 1
自動化內容策展                1
資訊檢索                   1
自動化新聞摘要                1
音樂自動標籤                 1
多目標學習                  1
